In [1]:
import numpy
import toyplot.text

x = 0
y = 0
text = "text"
style = {
    "font-size":"12px",
    "-toyplot-anchor-shift":"0",
    "text-anchor":"start",
    "alignment-baseline":"middle",
    "baseline-shift":"-50%",
}

left, right, top, bottom = toyplot.text.extents(x, y, text, style)
print left, right, top, bottom

[ 0.] [ 48.] [ 0.] [ 12.]
